In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt
from keras.layers import Input, concatenate, add, Multiply, Lambda
from keras.layers import Conv3D, MaxPooling3D, MaxPooling2D, UpSampling2D,UpSampling3D, Conv2D
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.models import Model
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.initializers import HeUniform
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Add, Multiply, BatchNormalization, Activation
from tensorflow.keras.models import Model

In [4]:
class Identity_block(tf.keras.Model):
    def __init__(self, filters, kernel_size, strides, padding = 'same'):
        super(Identity_block, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.act1 = tf.keras.layers.Activation('relu')
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding,strides = strides)

        self.bn2 = tf.keras.layers.BatchNormalization()
        self.act2 = tf.keras.layers.Activation('relu')
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size,padding=padding,strides = 1 )

        self.shortcut = tf.keras.layers.Conv2D(filters, kernel_size = (1,1), padding=padding, strides=strides)
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.add = tf.keras.layers.Add()

    def call(self, inputs):
        x = self.bn1(inputs)
        x = self.act1(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.conv2(x)
        shortcut = self.shortcut(inputs)
        shortcut = self.bn3(shortcut)
        x = self.add([x, shortcut])
        return x

class Upsample_Concatenate_block(tf.keras.Model):
    def __init__(self):
        super(Upsample_Concatenate_block, self).__init__()
        self.upsample = tf.keras.layers.UpSampling2D(size=(2, 2))
        self.concat = tf.keras.layers.Concatenate()

    def call(self, inputs, skip):
        x = self.upsample(inputs)
        x = self.concat([x, skip])
        return x

class Residual_Unet(tf.keras.Model):
    def __init__(self):
        super(Residual_Unet, self).__init__()

        # Encoder
        self.conv1 = tf.keras.layers.Conv2D(32, 3, padding='same',strides = 1)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.act1 = tf.keras.layers.Activation('relu')
        self.conv2 = tf.keras.layers.Conv2D(32, 3, padding='same', strides = 1)
        self.shortcut1 = tf.keras.layers.Conv2D(32, 1, padding='same',  strides = 1)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.add1 = tf.keras.layers.Add()
        # try different kernel sizes
        self.idblock1 = Identity_block(64, 3, 2)
        self.idblock2 = Identity_block(128, 3, 2)
        self.idblock3 = Identity_block(256, 3, 2)

        # bridge
        self.bridge = Identity_block(512, 3, 1)

        # Decoder
        self.up_concat1 = Upsample_Concatenate_block()
        self.idblock4 = Identity_block(256, 3, 1)

        self.up_concat2 = Upsample_Concatenate_block()
        self.idblock5 = Identity_block(128, 3, 1)

        self.up_concat3 = Upsample_Concatenate_block()
        self.idblock6 = Identity_block(64, 3, 1)
        # try different dense layers
        self.output_con_def = tf.keras.layers.Conv2D(2, 1, padding='same', activation='linear')


    def call(self, inputs):
        moving_image, fixed_image = inputs
        x = tf.concat([moving_image, fixed_image], axis=-1)

        # Encoder
        e0 = self.conv1(x)
        e0 = self.bn1(e0)
        e0 = self.act1(e0)
        e0 = self.conv2(e0)
        shortcut1 = self.shortcut1(e0)
        e0 = self.bn2(shortcut1)
        e0 = self.add1([e0, shortcut1])

        e1 = self.idblock1(e0)
        e2 = self.idblock2(e1)
        e3 = self.idblock3(e2)

        # bridge
        bridge = self.bridge(e3)

        # Decoder
        d1 = self.up_concat1(bridge, e2)
        d1 = self.idblock4(d1)

        d2 = self.up_concat2(d1, e1)
        d2 = self.idblock5(d2)

        d3 = self.up_concat3(d2, e0)
        d3 = self.idblock6(d3)

        output_def = self.output_con_def(d3)


        return output_def

fixed_input = Input(shape=(128, 128, 1), name="fixed_image")
moving_input = Input(shape=(128, 128, 1), name="moving_image")

resunet = Residual_Unet()
out_def = resunet([moving_input, fixed_input])

model = Model(inputs=[moving_input, fixed_input], outputs=out_def)

# Print model summary
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ moving_image        │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fixed_image         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residual__unet_1    │ (None, 128, 128,  │  7,831,970 │ moving_image[0][… │
│ (Residual_Unet)     │ 2)                │            │ fixed_image[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,831,970 (29.88 MB)

 Trainable params: 7,823,010 (29.84 MB)

 Non-trainable params: 8,960 (35.00 KB)